In [2]:
import yaml

In [17]:
yaml_file_path = '../db_creds.yaml'

# Read the YAML file and store its contents in a Python data structure (dictionary)
with open(yaml_file_path, 'r') as file:
    yaml_data = yaml.safe_load(file)

In [29]:
yaml_data

{'RDS_HOST': 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com',
 'RDS_PASSWORD': 'AiCore2022',
 'RDS_USER': 'aicore_admin',
 'RDS_DATABASE': 'postgres',
 'RDS_PORT': 5432}

In [30]:
from sqlalchemy import create_engine

In [31]:
DATABASE_TYPE = 'postgresql'
ENDPOINT = "<your-amazon-rds-endpoint>"
USER = yaml_data['RDS_USER']
PASSWORD = "<your-password>"
PORT = yaml_data['RDS_PORT']
DATABASE = yaml_data['RDS_DATABASE']

'postgres'

In [ ]:
sql_engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")